In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, LeaveOneOut, cross_val_predict, StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, recall_score, precision_score
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Input, Dropout

from tqdm import tqdm
from joblib import dump, load

In [2]:
df = pd.read_csv("heloc.csv")

In [3]:
df = df[~df.apply(lambda row: all(row[1:] < 0), axis=1)]

In [4]:
mapeo = {'Good': True, 'Bad': False}
df = df.replace(mapeo)

In [5]:
df

,RiskPerformance,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,...,PercentInstallTrades,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance
0,False,55,144,4,84,20,3,0,83,2,...,43,0,0,0,33,-8,8,1,1,69
1,False,61,58,15,41,2,4,4,100,-7,...,67,0,0,0,0,-8,0,-8,-8,0
2,False,67,66,5,24,9,0,0,100,-7,...,44,0,4,4,53,66,4,2,1,86
3,False,66,169,1,73,28,1,1,93,76,...,57,0,5,4,72,83,6,4,3,91
4,False,81,333,27,132,12,0,0,100,-7,...,25,0,1,1,51,89,3,1,0,80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10454,True,73,131,5,57,21,0,0,95,80,...,19,7,0,0,26,-8,5,2,0,100
10455,False,65,147,39,68,11,0,0,92,28,...,42,1,1,1,86,53,2,2,1,80
10456,False,74,129,6,64,18,1,1,100,-7,...,33,3,4,4,6,-8,5,-8,0,56
10457,False,72,234,12,113,42,2,2,96,35,...,20,6,0,0,19,-8,4,1,0,38


In [6]:
(df < 0).sum().sort_values(ascending=False)

MSinceMostRecentDelq                  4840
NetFractionInstallBurden              3419
MSinceMostRecentInqexcl7days          2331
NumInstallTradesWBalance               861
NumBank2NatlTradesWHighUtilization     583
MSinceOldestTradeOpen                  239
NetFractionRevolvingBurden             186
NumRevolvingTradesWBalance             156
PercentTradesWBalance                   18
ExternalRiskEstimate                    10
PercentInstallTrades                     0
NumInqLast6Mexcl7days                    0
NumInqLast6M                             0
RiskPerformance                          0
NumTradesOpeninLast12M                   0
MaxDelqEver                              0
MaxDelq2PublicRecLast12M                 0
PercentTradesNeverDelq                   0
NumTrades90Ever2DerogPubRec              0
NumTrades60Ever2DerogPubRec              0
NumSatisfactoryTrades                    0
AverageMInFile                           0
MSinceMostRecentTradeOpen                0
NumTotalTra

In [7]:
df_con_nans = df.replace([-9,-8,-7], np.nan)

## RandomForestClassifier

In [8]:
best_results = {"Modelo": None, "Precision": 0 ,"Accuracy": 0, "Recall": 0, "F1": 0, "AUC": 0, "Vecinos": 0}

for neighbors in tqdm(range(1, 21), desc="Progreso"):
    imputer = KNNImputer(n_neighbors=neighbors)
    df_imputed = pd.DataFrame(imputer.fit_transform(df_con_nans), columns=df_con_nans.columns)
    X = df_imputed.drop(["RiskPerformance"], axis=1)
    y = df_imputed["RiskPerformance"]

    model = RandomForestClassifier(random_state=42)

    cv_predictions = cross_val_predict(model, X, y, cv=5)

    avg_precision = precision_score(y, cv_predictions)
    avg_accuracy  = accuracy_score(y, cv_predictions)
    avg_recall    = recall_score(y, cv_predictions)
    avg_auc       = roc_auc_score(y, cv_predictions)
    avg_f1        = f1_score(y, cv_predictions)

    if avg_precision > best_results["Precision"]:
        best_results["Modelo"]    = model
        best_results["Precision"] = avg_precision
        best_results["Accuracy"]  = avg_accuracy
        best_results["Recall"]    = avg_recall
        best_results["F1"]        = avg_f1
        best_results["AUC"]       = avg_auc
        best_results["Vecinos"]   = neighbors

best_model = best_results["Modelo"]
best_model.fit(X, y)
feature_importance = best_model.feature_importances_
feature_names = X.columns

feature_importance_sorted = sorted(zip(feature_names, feature_importance), key=lambda x: x[1], reverse=True)
sorted_feature_names, sorted_feature_importance = zip(*feature_importance_sorted)

plt.bar(sorted_feature_names, sorted_feature_importance)
plt.xlabel('Feature Names')
plt.ylabel('Feature Importance')
plt.title('Feature Importance Plot (Sorted)')
plt.xticks(rotation=90)
plt.show()


dump(best_model, 'modelo_rfc.joblib')

print("Modelo:"   , best_results["Modelo"])
print("Vecinos:"  , best_results["Vecinos"])
print("Precision:", best_results["Precision"])
print("Recall:"   , best_results["Recall"])
print("F1 Score:" , best_results["F1"])
print("Accuracy:" , best_results["Accuracy"])
print("AUC:"      , best_results["AUC"])
print("Modelo guardado como 'modelo_rfc.joblib'")

Progreso: 100%|██████████| 20/20 [02:40<00:00,  8.01s/it]

Modelo: RandomForestClassifier(random_state=42)
Vecinos: 5
Precision: 0.7338872670110038
Recall: 0.6901795142555438
F1 Score: 0.7113626469307792
Accuracy: 0.7313342113261068
AUC: 0.7297276075950616
Modelo guardado como 'modelo_rfc.joblib'


## MLPClassifier

In [9]:
best_results = {"Modelo": None, "Precision": 0 ,"Accuracy": 0, "Recall": 0, "F1": 0, "AUC": 0, "Vecinos": 0}

imputer = KNNImputer()

for neighbors in tqdm(range(1, 21), desc="Progreso"):
    df_imputed = pd.DataFrame(imputer.fit_transform(df_con_nans), columns=df_con_nans.columns)
    X = df_imputed.drop(["RiskPerformance"], axis=1)
    y = df_imputed["RiskPerformance"]

    model = MLPClassifier(random_state=42, max_iter=500)
    
    cv_predictions = cross_val_predict(model, X, y, cv=5)
    
    avg_precision = precision_score(y, cv_predictions)
    avg_accuracy  = accuracy_score(y, cv_predictions)
    avg_recall    = recall_score(y, cv_predictions)
    avg_auc       = roc_auc_score(y, cv_predictions)
    avg_f1        = f1_score(y, cv_predictions)

    if avg_precision > best_results["Precision"]:
        best_results["Modelo"]    = model
        best_results["Precision"] = avg_precision
        best_results["Accuracy"]  = avg_accuracy
        best_results["Recall"]    = avg_recall
        best_results["F1"]        = avg_f1
        best_results["AUC"]       = avg_auc
        best_results["Vecinos"]   = neighbors

dump(best_results["Modelo"], 'modelo_red_neuronal_mlp.joblib')

print("Modelo:"   , best_results["Modelo"])
print("Vecinos:"  , best_results["Vecinos"])
print("Precision:", best_results["Precision"])
print("Recall:"   , best_results["Recall"])
print("F1 Score:" , best_results["F1"])
print("Accuracy:" , best_results["Accuracy"])
print("AUC:"      , best_results["AUC"])
print("Modelo guardado como 'modelo_red_neuronal_mlp.joblib'")

Progreso: 100%|██████████| 20/20 [05:12<00:00, 15.64s/it]

Modelo: MLPClassifier(max_iter=500, random_state=42)
Vecinos: 1
Precision: 0.6613021214337966
Recall: 0.7636747624076029
F1 Score: 0.7088111339802018
Accuracy: 0.6990173234727991
AUC: 0.7015414310480381
Modelo guardado como 'modelo_red_neuronal_mlp.joblib'


## Red StandardScaler

In [10]:
tf.random.set_seed(42)

best_results = {"Modelo": None, "Precision": 0, "Recall": 0,  "Accuracy": 0, "F1_Score": 0, "AUC": 0, "Vecinos": 0}

imputer = KNNImputer()

for neighbors in tqdm(range(1, 21), desc="Progreso"):
    df_imputed = pd.DataFrame(imputer.fit_transform(df_con_nans), columns=df_con_nans.columns)
    X = df_imputed.drop("RiskPerformance", axis=1)
    y = df_imputed["RiskPerformance"]

    X_scaled = StandardScaler().fit_transform(X)

    input_nodes  = X_scaled.shape[1]
    output_nodes = 1

    model = Sequential()
    model.add(Input((input_nodes,)))
    model.add(Dense(64, activation="relu"))
    model.add(Dropout(0.3, seed=0))
    model.add(Dense(32, activation="relu"))
    model.add(Dropout(0.3, seed=1))
    model.add(Dense(16, activation="relu"))
    model.add(Dropout(0.3, seed=2))
    model.add(Dense(output_nodes))
    model.compile(optimizer="adam", loss="mean_squared_logarithmic_error")

    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

    precision_scores, recall_scores, f1_scores, auc_scores, accuracy_scores = [], [], [], [], []

    for train_index, test_index in skf.split(X_scaled, y):
        X_train, X_test = X_scaled[train_index], X_scaled[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    
        history = model.fit(
            X_train,
            y_train,
            epochs=100,
            validation_data=(X_test, y_test),
            verbose=2
        )
    
        y_pred = (model.predict(X_test) > 0.5).astype(int)
    
        precision = precision_score(y_test, y_pred)
        recall    = recall_score(y_test, y_pred)
        f1        = f1_score(y_test, y_pred)
        auc       = roc_auc_score(y_test, y_pred)
        accuracy  = accuracy_score(y_test, y_pred)
    
        precision_scores.append(precision)
        recall_scores.append(recall)
        f1_scores.append(f1)
        auc_scores.append(auc)
        accuracy_scores.append(accuracy)

        avg_precision = np.mean(precision_scores)
        avg_recall    = np.mean(recall_scores)
        avg_f1        = np.mean(f1_scores)
        avg_auc       = np.mean(auc_scores)
        avg_accuracy  = np.mean(accuracy_scores)

    if avg_f1 > best_results["F1_Score"]:
        best_results["Modelo"]    = model
        best_results["Precision"] = avg_precision
        best_results["Accuracy"]  = avg_accuracy
        best_results["Recall"]    = avg_recall
        best_results["F1_Score"]  = avg_f1
        best_results["AUC"]       = avg_auc
        best_results["Vecinos"]   = neighbors
        
best_results["Modelo"].save('modelo_red_neuronal_seq.h5')

print("Modelo:"   , best_results["Modelo"])
print("Vecinos:"  , best_results["Vecinos"])
print("Precision:", best_results["Precision"])
print("Recall:"   , best_results["Recall"])
print("F1 Score:" , best_results["F1_Score"])
print("Accuracy:" , best_results["Accuracy"])
print("AUC:"      , best_results["AUC"])
print("Modelo guardado como 'modelo_red_neuronal_seq.h5'")

Progreso:   0%|          | 0/20 [00:00<?, ?it/s]



Epoch 1/100

247/247 - 1s - loss: 0.1283 - val_loss: 0.0960 - 837ms/epoch - 3ms/step
Epoch 2/100
247/247 - 0s - loss: 0.1018 - val_loss: 0.0900 - 208ms/epoch - 842us/step
Epoch 3/100
247/247 - 0s - loss: 0.0983 - val_loss: 0.0885 - 206ms/epoch - 834us/step
Epoch 4/100
247/247 - 0s - loss: 0.0957 - val_loss: 0.0882 - 211ms/epoch - 855us/step
Epoch 5/100
247/247 - 0s - loss: 0.0956 - val_loss: 0.0891 - 210ms/epoch - 852us/step
Epoch 6/100
247/247 - 0s - loss: 0.0935 - val_loss: 0.0879 - 216ms/epoch - 875us/step
Epoch 7/100
247/247 - 0s - loss: 0.0932 - val_loss: 0.0869 - 215ms/epoch - 871us/step
Epoch 8/100
247/247 - 0s - loss: 0.0930 - val_loss: 0.0860 - 214ms/epoch - 865us/step
Epoch 9/100
247/247 - 0s - loss: 0.0915 - val_loss: 0.0874 - 216ms/epoch - 873us/step
Epoch 10/100
247/247 - 0s - loss: 0.0923 - val_loss: 0.0862 - 208ms/epoch - 844us/step
Epoch 11/100
247/247 - 0s - loss: 0.0920 - val_loss: 0.0859 - 216ms/epoch - 873us/step
Epoch 12/100
247/247 - 0s - loss: 0.0921 - val_loss

Progreso:   5%|▌         | 1/20 [01:49<34:41, 109.57s/it]

Epoch 1/100
247/247 - 1s - loss: 0.1503 - val_loss: 0.0924 - 649ms/epoch - 3ms/step
Epoch 2/100
247/247 - 0s - loss: 0.1089 - val_loss: 0.0914 - 204ms/epoch - 827us/step
Epoch 3/100
247/247 - 0s - loss: 0.1011 - val_loss: 0.0899 - 207ms/epoch - 838us/step
Epoch 4/100
247/247 - 0s - loss: 0.0980 - val_loss: 0.0880 - 206ms/epoch - 836us/step
Epoch 5/100
247/247 - 0s - loss: 0.0971 - val_loss: 0.0900 - 212ms/epoch - 859us/step
Epoch 6/100
247/247 - 0s - loss: 0.0946 - val_loss: 0.0880 - 207ms/epoch - 837us/step
Epoch 7/100
247/247 - 0s - loss: 0.0933 - val_loss: 0.0873 - 206ms/epoch - 833us/step
Epoch 8/100
247/247 - 0s - loss: 0.0930 - val_loss: 0.0873 - 205ms/epoch - 830us/step
Epoch 9/100
247/247 - 0s - loss: 0.0923 - val_loss: 0.0884 - 207ms/epoch - 839us/step
Epoch 10/100
247/247 - 0s - loss: 0.0933 - val_loss: 0.0874 - 204ms/epoch - 826us/step
Epoch 11/100
247/247 - 0s - loss: 0.0920 - val_loss: 0.0868 - 207ms/epoch - 837us/step
Epoch 12/100
247/247 - 0s - loss: 0.0911 - val_loss: 0

Progreso:  10%|█         | 2/20 [03:39<32:53, 109.62s/it]

...

Progreso:  95%|█████████▌| 19/20 [34:48<01:50, 110.43s/it]

Epoch 1/100
247/247 - 1s - loss: 0.1381 - val_loss: 0.1000 - 680ms/epoch - 3ms/step
Epoch 2/100
247/247 - 0s - loss: 0.1068 - val_loss: 0.0912 - 211ms/epoch - 855us/step
Epoch 3/100
247/247 - 0s - loss: 0.1014 - val_loss: 0.0893 - 215ms/epoch - 871us/step
Epoch 4/100
247/247 - 0s - loss: 0.0971 - val_loss: 0.0871 - 214ms/epoch - 866us/step
Epoch 5/100
247/247 - 0s - loss: 0.0967 - val_loss: 0.0883 - 212ms/epoch - 858us/step
Epoch 6/100
247/247 - 0s - loss: 0.0947 - val_loss: 0.0883 - 213ms/epoch - 862us/step
Epoch 7/100
247/247 - 0s - loss: 0.0945 - val_loss: 0.0872 - 218ms/epoch - 884us/step
Epoch 8/100
247/247 - 0s - loss: 0.0935 - val_loss: 0.0865 - 213ms/epoch - 861us/step
Epoch 9/100
247/247 - 0s - loss: 0.0930 - val_loss: 0.0881 - 215ms/epoch - 869us/step
Epoch 10/100
247/247 - 0s - loss: 0.0928 - val_loss: 0.0870 - 208ms/epoch - 843us/step
Epoch 11/100
247/247 - 0s - loss: 0.0920 - val_loss: 0.0860 - 211ms/epoch - 855us/step
Epoch 12/100
247/247 - 0s - loss: 0.0921 - val_loss: 0

Progreso: 100%|██████████| 20/20 [36:39<00:00, 109.97s/it]

Modelo: <keras.src.engine.sequential.Sequential object at 0x0000023FBC9C8250>
Vecinos: 6
Precision: 0.7650913147232197
Recall: 0.712143611404435
F1 Score: 0.7375278778146557
Accuracy: 0.757170302540623
AUC: 0.7554106680908754
Modelo guardado como 'modelo_red_neuronal_seq.h5'



C:\Users\Adri1\anaconda3\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [11]:
top_model = load_model('modelo_red_neuronal_seq.h5')

In [12]:
top_model

# Carga Modelos

In [13]:
load('modelo_rfc.joblib')

RandomForestClassifier(random_state=42)

In [14]:
load('modelo_red_neuronal_mlp.joblib')

MLPClassifier(max_iter=500, random_state=42)

In [15]:
load_model('modelo_red_neuronal_seq.h5')